In [30]:
import bw2data

In [78]:
from enbios2.generic.files import DataPath

base_folder = DataPath("temp/alex_enb2")

In [79]:
from enbios2.bw2.util import report

report()

Project: default
[]
Project: ecoinvent
['biosphere3', 'cutoff_3.9.1_default', 'seeds']
Project: excel_import
['biosphere3', 'csv_db']


In [80]:
import openpyxl

excel_file = base_folder / "base_file_simplified2.xlsx"
workbook = openpyxl.load_workbook(excel_file)
processors = workbook["BareProcessors simulation"]

In [81]:
# collect processor, carrier and CODE
processors_data = []
for row in processors.iter_rows(min_row=2):
    if row[1].value:
        processors_data.append((row[1].value, row[2].value, row[9].value))

# COPY, IN ORDER TO CHECK WHICH ARE MISSING
remaining = processors_data.copy()

processors_data

[('wind_onshore', 'electricity', '81174ec2c20931c1a36f65c654bbd11e'),
 ('wind_offshore', 'electricity', '46839adbc3cf15621ddbbe687eaa6df1'),
 ('hydro_run_of_river', 'electricity', 'b870e3d3ddd5b634d016940064b27532'),
 ('hydro_reservoir', 'electricity', 'c868c4688fbf78f5ca3787ac3d83312b'),
 ('ccgt', 'electricity', '4833b6352dfe15c95ae46fd280371cd3'),
 ('chp_biofuel_extraction', 'electricity', '1952fb157a18463b0028629917057bcf'),
 ('open_field_pv', 'electricity', '206c2e2c30f45d47ea2e9a3701f8ecc5'),
 ('electrolysis', 'electricity', 'FILL'),
 ('existing_wind', 'electricity', '81174ec2c20931c1a36f65c654bbd11e'),
 ('existing_pv', 'electricity', '206c2e2c30f45d47ea2e9a3701f8ecc5'),
 ('roof_mounted_pv', 'electricity', '1d7a3591e7a3d033a25e0eaf699bb50b'),
 ('chp_wte_back_pressure', 'electricity', 'fd6356e68720aec505c69cdadb3046fc'),
 ('chp_methane_extraction', 'electricity', 'a0d0ab65435d0487d33b4be142483d76'),
 ('waste_supply', 'electricity', 'fd6356e68720aec505c69cdadb3046fc'),
 ('biofuel_su

In [84]:
bw2data.projects.set_current("ecoinvent")
list(bw2data.databases)

['biosphere3', 'cutoff_3.9.1_default', 'seeds']

In [85]:
from bw2data.errors import UnknownObject

# read all values in column I
db = bw2data.Database("cutoff_3.9.1_default")
activities = []

for pd in processors_data:
    try:
        activity = db.get_node(pd[2])
        activities.append((activity, *pd))
    except UnknownObject:
        print(pd, "not found")

('electrolysis', 'electricity', 'FILL') not found


In [86]:
activities

[('electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, PT, None),
  'wind_onshore',
  'electricity',
  '81174ec2c20931c1a36f65c654bbd11e'),
 ('electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, PT, None),
  'wind_offshore',
  'electricity',
  '46839adbc3cf15621ddbbe687eaa6df1'),
 ('electricity production, hydro, run-of-river' (kilowatt hour, PT, None),
  'hydro_run_of_river',
  'electricity',
  'b870e3d3ddd5b634d016940064b27532'),
 ('electricity production, hydro, reservoir, non-alpine region' (kilowatt hour, PT, None),
  'hydro_reservoir',
  'electricity',
  'c868c4688fbf78f5ca3787ac3d83312b'),
 ('electricity production, natural gas, combined cycle power plant' (kilowatt hour, PT, None),
  'ccgt',
  'electricity',
  '4833b6352dfe15c95ae46fd280371cd3'),
 ('heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, PT, None),
  'chp_biofuel_extraction',
  'electricity',
  '1952fb157a18463b0028629917057bcf'),
 ('electri

In [87]:
from collections import Counter

potential_aliases = [act[1] for act in activities]
print("All unique:", len(potential_aliases) == len(set(potential_aliases)))


All unique: False


In [88]:
# user a counter, and redo the array of activities, by adding a string the beginning with a new full unique (hopefully) alias
counts = Counter(potential_aliases)

activities_ = []
for act in activities:
    full_alias = f"{act[1]}_{act[2]}"
    activities_.append((full_alias, *act[1:]))

# check if its all good (should give False)
any((v > 1 for v in Counter([act[0] for act in activities_]).values()))

False

In [89]:
from ipywidgets import HTML
import tabulate

activities = activities_

table = tabulate.tabulate(activities, headers=["full alias [0]", "orig alias", "carrier", "code [3]"], tablefmt='html')
display(HTML(table))

HTML(value='<table>\n<thead>\n<tr><th>full alias [0]                    </th><th>orig alias            </th><t…

In [90]:
from bw2data.backends import Activity

activities_bw: dict[str, Activity] = {}
for activity in activities:
    activities_bw[activity[0]] = db.get_node(activity[3])

activities_bw

{'wind_onshore_electricity': 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, PT, None),
 'wind_offshore_electricity': 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, PT, None),
 'hydro_run_of_river_electricity': 'electricity production, hydro, run-of-river' (kilowatt hour, PT, None),
 'hydro_reservoir_electricity': 'electricity production, hydro, reservoir, non-alpine region' (kilowatt hour, PT, None),
 'ccgt_electricity': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, PT, None),
 'chp_biofuel_extraction_electricity': 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, PT, None),
 'open_field_pv_electricity': 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' (kilowatt hour, PT, None),
 'existing_wind_electricity': 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, PT, None),
 'existing_pv_electricity': 'electricity 

In [93]:
SEEDS_DB_NAME = "seeds"
seeds_db = bw2data.Database(SEEDS_DB_NAME)
if SEEDS_DB_NAME in bw2data.databases:
    bw2data.Database(SEEDS_DB_NAME).delete_instance()
seeds_db = bw2data.Database(SEEDS_DB_NAME)
seeds_db.register()

In [94]:
from tqdm import tqdm


# from enbios2.bw2.util import full_duplicate

def full_duplicate(activity: Activity, code=None, **kwargs) -> Activity:
    """
    Make a copy of an activity with its upstream exchanges
    (Otherwise, you cannot calculate the lca of the copy)
    :param activity: the activity to copy
    :param code: code of the new activity
    :param kwargs: other data for the copy
    :return: new activity
    """
    activity_copy = activity.copy(code, **kwargs)
    for upstream in activity.upstream():
        upstream.output.new_exchange(input=activity_copy, type=upstream["type"], amount=upstream.amount).save()
    activity_copy.save()
    return activity_copy


for alias, activity in activities_bw.items():
    clone1 = full_duplicate(activity)
    clone1["name"] = clone1["name"] + "__PRT_1"
    clone1["database"] = SEEDS_DB_NAME
    clone1["alias"] = alias + "__PRT_1"
    clone1.save()

    clone2 = full_duplicate(activity)
    clone2["name"] = clone2["name"] + "__PRT_2"
    clone2["database"] = SEEDS_DB_NAME
    clone2["alias"] = alias + "__PRT_2"
    clone2.save()

Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity dataset to database `seeds`
Successfully switch activity da

In [95]:
list(seeds_db)

['market for biomethane, high pressure__PRT_2' (cubic meter, CH, None),
 'electricity production, hydro, run-of-river__PRT_1' (kilowatt hour, PT, None),
 'electricity production, natural gas, combined cycle power plant__PRT_2' (kilowatt hour, PT, None),
 'electricity production, hydro, pumped storage__PRT_1' (kilowatt hour, PT, None),
 'electricity, from municipal waste incineration to generic market for electricity, medium voltage__PRT_1' (kilowatt hour, PT, None),
 'heat, from municipal waste incineration to generic market for heat district or industrial, other than natural gas__PRT_1' (megajoule, PT, None),
 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014__PRT_2' (megajoule, PT, None),
 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014__PRT_1' (megajoule, PT, None),
 'market for heat storage, 2000l__PRT_2' (unit, GLO, None),
 'electricity production, hydro, run-of-river__PRT_2' (kilowatt hour, PT, None),
 'heat and power co-generatio

In [96]:
enbios2_activities = {
    activity["alias"]: {
        "id": {"code": activity["code"]}
    }
    for activity in list(seeds_db)
}

In [106]:
enbios2_activities.keys()

dict_keys(['biofuel_to_methanol_methanol__PRT_1', 'ccgt_electricity__PRT_2', 'Methane_storage_methane__PRT_1', 'hydro_reservoir_electricity__PRT_2', 'biofuel_to_methane_methane__PRT_1', 'existing_pv_electricity__PRT_2', 'existing_wind_electricity__PRT_2', 'chp_methane_extraction_electricity__PRT_1', 'el_import_electricity__PRT_2', 'chp_methane_extraction_heat__PRT_2', 'pumped_hydro_electricity__PRT_2', 'existing_pv_electricity__PRT_1', 'open_field_pv_electricity__PRT_1', 'heat_storage_small_heat__PRT_2', 'methane_boiler_heat__PRT_2', 'biofuel_boiler_heat__PRT_2', 'biofuel_supply_biofuel__PRT_1', 'open_field_pv_electricity__PRT_2', 'roof_mounted_pv_electricity__PRT_2', 'biofuel_supply_biofuel__PRT_2', 'chp_methane_extraction_heat__PRT_1', 'biofuel_to_methanol_methanol__PRT_2', 'chp_biofuel_extraction_heat__PRT_1', 'chp_biofuel_extraction_heat__PRT_2', 'biofuel_to_diesel_diesel__PRT_1', 'existing_wind_electricity__PRT_1', 'wind_offshore_electricity__PRT_1', 'pumped_hydro_electricity__PRT

In [98]:
# in case things got messed up:
# seeds_db.delete_instance()

In [111]:
from csv import DictReader
from enbios2.generic.files import ReadDataPath

enbios2scenarios = []
raw_scenarios = ReadDataPath(base_folder / "data.csv")  # .read_data()
raw_scenario_data = list(DictReader(raw_scenarios.open(encoding="utf-8"), delimiter=";"))
raw_scenario_data[:5]

[{'Scenarios': '0_',
  'year': '2012',
  'techs': 'battery',
  'locs': 'PRT_1',
  'carriers': 'electricity',
  'unit': 'Twh',
  'flow_out_sum': '0.000525631'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'battery',
  'locs': 'PRT_2',
  'carriers': 'electricity',
  'unit': 'Twh',
  'flow_out_sum': '0.000575126'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'biofuel_boiler',
  'locs': 'PRT_1',
  'carriers': 'heat',
  'unit': 'Twh',
  'flow_out_sum': '8.9698237'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'biofuel_boiler',
  'locs': 'PRT_2',
  'carriers': 'heat',
  'unit': 'Twh',
  'flow_out_sum': '0.000386907'},
 {'Scenarios': '0_',
  'year': '2012',
  'techs': 'biofuel_supply',
  'locs': 'PRT_1',
  'carriers': 'biofuel',
  'unit': 'Twh',
  'flow_out_sum': '20.39066515'}]

In [112]:
current_scenario_index = 0
current_scenario = {}
for row in raw_scenario_data:
    scenario_index = int(row["Scenarios"][:-1])
    assert current_scenario_index <= scenario_index <= current_scenario_index + 1
    if scenario_index == current_scenario_index + 1:
        current_scenario_index = scenario_index
        enbios2scenarios.append({"activities": current_scenario.copy()})
        current_scenario = {}
    alias = f'{row["techs"]}_{row["carriers"]}__{row["locs"]}'
    if alias not in enbios2_activities:
        print("skipping", alias)
        continue
    else:
        print("adding", alias)
    current_scenario[alias] = [row["unit"], row["flow_out_sum"]]

enbios2scenarios[0]

skipping battery_electricity__PRT_1
skipping battery_electricity__PRT_2
adding biofuel_boiler_heat__PRT_1
adding biofuel_boiler_heat__PRT_2
adding biofuel_supply_biofuel__PRT_1
adding biofuel_supply_biofuel__PRT_2
adding biofuel_to_diesel_diesel__PRT_1
adding biofuel_to_diesel_diesel__PRT_2
skipping biofuel_to_liquids_diesel__PRT_1
skipping biofuel_to_liquids_electricity__PRT_1
skipping biofuel_to_liquids_kerosene__PRT_1
skipping biofuel_to_liquids_diesel__PRT_2
skipping biofuel_to_liquids_electricity__PRT_2
skipping biofuel_to_liquids_kerosene__PRT_2
adding biofuel_to_methane_methane__PRT_1
adding biofuel_to_methane_methane__PRT_2
adding biofuel_to_methanol_methanol__PRT_1
adding biofuel_to_methanol_methanol__PRT_2
adding ccgt_electricity__PRT_1
adding ccgt_electricity__PRT_2
adding chp_biofuel_extraction_electricity__PRT_1
adding chp_biofuel_extraction_heat__PRT_1
adding chp_biofuel_extraction_electricity__PRT_2
adding chp_biofuel_extraction_heat__PRT_2
skipping chp_hydrogen_electric

{'activities': {'biofuel_boiler_heat__PRT_1': ['Twh', '8.9698237'],
  'biofuel_boiler_heat__PRT_2': ['Twh', '0.000386907'],
  'biofuel_supply_biofuel__PRT_1': ['Twh', '20.39066515'],
  'biofuel_supply_biofuel__PRT_2': ['Twh', '16.11178774'],
  'biofuel_to_diesel_diesel__PRT_1': ['Twh', '1.075645078'],
  'biofuel_to_diesel_diesel__PRT_2': ['Twh', '1.014019571'],
  'biofuel_to_methane_methane__PRT_1': ['Twh', '4.349878397'],
  'biofuel_to_methane_methane__PRT_2': ['Twh', '3.476858228'],
  'biofuel_to_methanol_methanol__PRT_1': ['Twh', '0.761035396'],
  'biofuel_to_methanol_methanol__PRT_2': ['Twh', '6.14514345'],
  'ccgt_electricity__PRT_1': ['Twh', '2.01E-05'],
  'ccgt_electricity__PRT_2': ['Twh', '1.39E-05'],
  'chp_biofuel_extraction_electricity__PRT_1': ['Twh', '8.59E-05'],
  'chp_biofuel_extraction_heat__PRT_1': ['Twh', '0.000144763'],
  'chp_biofuel_extraction_electricity__PRT_2': ['Twh', '6.97E-05'],
  'chp_biofuel_extraction_heat__PRT_2': ['Twh', '0.000126529'],
  'chp_methane_ex

In [113]:
from ast import literal_eval

methods = workbook["ScalarIndicators"]

enbios2_methods = {}

for row in methods.iter_rows(min_row=2):
    if row[5].value:
        method_tuple = literal_eval(row[5].value)
        enbios2_methods[method_tuple[2]] = method_tuple

enbios2_methods

{'ozone depletion potential (ODPinfinite)': ('ReCiPe 2016 v1.03, midpoint (H)',
  'ozone depletion',
  'ozone depletion potential (ODPinfinite)'),
 'agricultural land occupation (LOP)': ('ReCiPe 2016 v1.03, midpoint (H)',
  'land use',
  'agricultural land occupation (LOP)'),
 'surplus ore potential (SOP)': ('ReCiPe 2016 v1.03, midpoint (H)',
  'material resources: metals/minerals',
  'surplus ore potential (SOP)'),
 'global warming potential (GWP1000)': ('ReCiPe 2016 v1.03, midpoint (H)',
  'climate change',
  'global warming potential (GWP1000)')}

In [114]:
enbios2_data = {
    "bw_project": "ecoinvent",
    "activities": enbios2_activities,
    "methods": enbios2_methods,
    "scenarios": enbios2scenarios[:1]
}

In [115]:
from enbios2.base.experiment import Experiment
from enbios2.models.experiment_models import ExperimentData

exp_data = ExperimentData(**enbios2_data)
exp = Experiment(exp_data)

2023-07-13 13:01:32,343 - enbios2.base.experiment - DEBUG - activity dict
2023-07-13 13:01:34,143 - enbios2.base.experiment - ERROR - Cannot parse output unit 'Twh'- of activity ExtendedExperimentActivityData(id=ExperimentActivityId(database=None, code='dbc74e6b55654a64b9e7b3526c13d44b', name='market for ethanol, without water, in 99.7% solution state, from fermentation, vehicle grade__PRT_1', location='CH', unit='kilogram', alias='biofuel_supply_biofuel__PRT_1'), output=None, orig_id=ExperimentActivityId(database=None, code='dbc74e6b55654a64b9e7b3526c13d44b', name=None, location=None, unit=None, alias=None), bw_activity='market for ethanol, without water, in 99.7% solution state, from fermentation, vehicle grade__PRT_1' (kilogram, CH, None), scenario_outputs=None). Cannot convert from 'terawatt_hour' ([length] ** 2 * [mass] / [time] ** 2) to 'kilogram' ([mass]). Consider the unit definition to 'enbios2/base/unit_registry.py'


Exception: Unit error, Cannot convert from 'terawatt_hour' ([length] ** 2 * [mass] / [time] ** 2) to 'kilogram' ([mass]); For activity: ExperimentActivityId(database=None, code='dbc74e6b55654a64b9e7b3526c13d44b', name='market for ethanol, without water, in 99.7% solution state, from fermentation, vehicle grade__PRT_1', location='CH', unit='kilogram', alias='biofuel_supply_biofuel__PRT_1')

In [ ]:
from pathlib import Path

Path("results").mkdir(exist_ok=True)

for scenario in exp.scenarios:
    print(scenario.alias)
    result_tree = scenario.run()
    scenario.results_to_csv(Path(f"results/{scenario.alias}.csv"), True)